# Neighborhoods in Toronto

In this notebook, we explore neighborhoods in Toronto.

## Load Libraries

In [77]:
# !pip install beautifulsoup4 # uncomment to install beautifulsoup4 library

from bs4 import BeautifulSoup, element
import folium
from geopy.geocoders import Nominatim
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import urllib.request
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from sklearn.cluster import KMeans

## Scrape Wikipedia Page

The first step is to obtain the data. We start by scraping the list of postal codes in the Toronto area for postal codes, boroughs, and neighborhoods.

### Details
Each entry in the table is contained inside the `p` element so we use Beautiful Soup to find all such `p` elements and process them. The first two don't belong to the table so we omit them.

If there is an area specification such as Central, Northwest, or South whether meant to describe a borough or a neighborhood, it is omitted. For example, M3N has North York (Downsview) Northwest.

If the detailed information is some sort of enclave, e.g., M7R, then all information aside from the borough is omitted. That is, we treat it like a borough without a neighborhood.

In [6]:
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as response:
    html = response.read()

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [51]:
data = []
enclave_postal_codes = ["M5W", "M7R", "M7Y"] # Enclaves
excluded_modifiers = ["Northwest", "West", "Central", "East", "North", "South"]
for line in soup.find_all('p')[3:]:
    postal_code, _, span, _ = line.children

    def extract_text(x):
        if isinstance(x, element.Tag):
            x = x.get_text()
        return x.strip()
    
    # If the length of the list is 1, then the postal code is not assigned.
    children = list(span.children)
    n = len(children)
    postal_code = extract_text(postal_code)
    
    if n > 1:
        processed_text = []
        inside_parentheses = False
        for i, x in enumerate(children): 
            text = extract_text(x)
            if x.name != "br":
                processed_text.append(text)
                
                if text.startswith("("):
                    inside_parentheses = True
                elif text.endswith(")"):
                    inside_parentheses = False
                    
            elif x.name == "br":
                if not inside_parentheses:
                    processed_text.append("|")
            
        text_str = " ".join(processed_text)
        snippets = [x.strip() for x in text_str.split("|")
                    if x.strip() not in excluded_modifiers]

        if postal_code not in enclave_postal_codes:
            for i, snippet in enumerate(snippets):
                if not snippet.startswith("("):
                    if i+1 < len(snippets) and snippets[i+1].startswith("("):
                        borough = snippet
                        neighborhoods_str = snippets[i+1][1:-1].strip().replace(" / ", ", ").replace("  ", " ")
                    else:
                        borough = snippet
                        neighborhoods_str = borough
                    data.append([postal_code, borough, neighborhoods_str])
        else:
            borough = snippets[0]
            neighborhoods_str = borough
            data.append([postal_code, borough, neighborhoods_str])

df = pd.DataFrame(columns=["Postal Code", "Borough", "kNeighborhood"], data=data)

In [52]:
# Check the head.

df.head()

,Postal Code,Borough,kNeighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Retrieve Postal Codes

We try to get the latitude and longitude for one of the postal codes using the code on 

In [11]:
"""# !pip install geocoder

import geocoder # import geocoder

def get_lat_long(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return(latitude, longitude)

get_lat_long("M3A")"""

'# !pip install geocoder\n\nimport geocoder # import geocoder\n\ndef get_lat_long(postal_code):\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google(\'{}, Toronto, Ontario\'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    latitude = lat_lng_coords[0]\n    longitude = lat_lng_coords[1]\n    return(latitude, longitude)\n\nget_lat_long("M3A")'

In [54]:
lat_long = pd.read_csv("Geospatial_Coordinates.csv")

lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
neighborhoods = df.merge(lat_long)

neighborhoods.head()

,Postal Code,Borough,kNeighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Visualization

In [56]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# We determine a slight shift to the latitude to get a good looking map.
latitude_shift = 0.065

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [58]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude+latitude_shift, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['kNeighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [59]:
# Restrict the data to Boroughs with the word Toronto.
tb_data = neighborhoods[neighborhoods["Borough"].map(lambda x: "Toronto" in x)]

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude+latitude_shift, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(tb_data['Latitude'], tb_data['Longitude'], tb_data['Borough'], tb_data['kNeighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Collection of venues and Analysis of neighborhoods

Having restricted our list of neighborhoods, we use the FourSquare API to retrieve nearby venues and run some analysis on the nearby regions. The code here comes from the Week 3, with the exception of a few minor edits. One such edit is the use of "kNeighborhoods" as a column name, because "Neighborhoods" is already a category.

In [60]:
# Load Credentials from a file
with open("secret.json", "r") as f:
    secret = json.load(f)
CLIENT_ID = secret["CLIENT_ID"]
CLIENT_SECRET =secret["CLIENT_SECRET"]
VERSION = '20180605'
LIMIT = 100

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            results = []
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['kNeighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
tb_venues = getNearbyVenues(names=tb_data['kNeighborhood'],
                            latitudes=tb_data['Latitude'],
                            longitudes=tb_data['Longitude']
                           )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District , Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Downtown To

In [63]:
tb_venues.shape

(1691, 7)

In [64]:
tb_venues.groupby('kNeighborhood').agg({"kNeighborhood": "count"})

,kNeighborhood
kNeighborhood,
Berczy Park,55
"Brockton, Parkdale Village, Exhibition Place",24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17
Central Bay Street,78
Christie,17
Church and Wellesley,83
"Commerce Court, Victoria Hotel",100
Davisville,34
Davisville North,8


In [65]:
print('There are {} uniques categories.'.format(len(tb_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [66]:
# one hot encoding
tb_onehot = pd.get_dummies(tb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tb_onehot['kNeighborhood'] = tb_venues['kNeighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tb_onehot.columns[-1]] + list(tb_onehot.columns[:-1])
tb_onehot = tb_onehot[fixed_columns]

tb_onehot.head()

,kNeighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [67]:
tb_onehot.shape

(1691, 234)

In [68]:
tb_grouped = tb_onehot.groupby('kNeighborhood').mean().reset_index()

tb_grouped.head()

,kNeighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.018182,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.036364,0.0,0.000000,0.0,0.018182,0.018182,0.0,0.036364,0.0,0.0,0.018182,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.018182,0.036364,0.0,0.0,0.000000,0.0,0.036364,0.000000,0.0,0.0,0.000000,0.0,0.054545,0.090909,0.0,0.0,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.000000,0.018182,0.0,0.0,0.018182,0.0,0.0,0.0,0.0,0.018182,0.000000,0.018182,0.000000,0.0,0.0,0.0,0.000000,0.0,0.018182,0.0,0.0,0.0,0.0,0.000000,0.036364,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0,0.0,0.018182,0.018182,0.000000,0.0,0.0,0.000

In [69]:
tb_grouped.shape

(40, 234)

In [70]:
num_top_venues = 5

for hood in tb_grouped['kNeighborhood']:
    print("----"+hood+"----")
    temp = tb_grouped[tb_grouped['kNeighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3  Farmers Market  0.04
4            Café  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2     Coffee Shop  0.08
3       Nightclub  0.08
4     Yoga Studio  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1   Italian Restaurant  0.05
2      Thai Restaurant  0.04
3         Burger Joint  0.04
4  Japanese Restaurant  0.04


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     Restaurant  0.06
4    Gas Station  0.06


--

In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['kNeighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['kNeighborhood'] = tb_grouped['kNeighborhood']

for ind in np.arange(tb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,kNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Yoga Studio,Convenience Store,Restaurant,Stadium,Italian Restaurant,Intersection
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Rental Car Location
3,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Burger Joint,Sandwich Place,Thai Restaurant,Gym / Fitness Center,Department Store,Middle Eastern Restaurant,Ice Cream Shop
4,Christie,Grocery Store,Café,Park,Coffee Shop,Gas Station,Restaurant,Candy Store,Baby Store,Nightclub,Italian Restaurant


## Clustering of Neighborhoods

Finally we run a k-means clustering algorithm to cluster the neighborhoods.

In [93]:
# set number of clusters
kclusters = 5

tb_grouped_clustering = tb_grouped.drop('kNeighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4,
       1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [94]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tb_merged = tb_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tb_merged = tb_merged.join(neighborhoods_venues_sorted.set_index('kNeighborhood'), on='kNeighborhood')

tb_merged.head() # check the last columns!

,Postal Code,Borough,kNeighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Restaurant,Theater,Bakery,Mexican Restaurant,Breakfast Spot,Café,Spa
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Yoga Studio,Burrito Place,Indian Restaurant,Beer Bar,Italian Restaurant,Nightclub,Juice Bar,Seafood Restaurant,Sandwich Place
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Bookstore,Italian Restaurant,Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Clothing Store,Beer Bar,Diner,Bakery,Breakfast Spot
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Neighborhood,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tb_merged['Latitude'], tb_merged['Longitude'], tb_merged['kNeighborhood'], tb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [100]:
tb_merged.loc[tb_merged['Cluster Labels'] == 0, tb_merged.columns[[1] + list(range(5, tb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,East Toronto,0,Convenience Store,Park,Coffee Shop,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
92,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [102]:
tb_merged.loc[tb_merged['Cluster Labels'] == 2, tb_merged.columns[[1] + list(range(5, tb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [103]:
tb_merged.loc[tb_merged['Cluster Labels'] == 3, tb_merged.columns[[1] + list(range(5, tb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [104]:
tb_merged.loc[tb_merged['Cluster Labels'] == 4, tb_merged.columns[[1] + list(range(5, tb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,Central Toronto,4,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [101]:
tb_merged.loc[tb_merged['Cluster Labels'] == 1, tb_merged.columns[[1] + list(range(5, tb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Park,Pub,Restaurant,Theater,Bakery,Mexican Restaurant,Breakfast Spot,Café,Spa
4,Downtown Toronto,1,Coffee Shop,Yoga Studio,Burrito Place,Indian Restaurant,Beer Bar,Italian Restaurant,Nightclub,Juice Bar,Seafood Restaurant,Sandwich Place
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Bookstore,Italian Restaurant,Restaurant
15,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Clothing Store,Beer Bar,Diner,Bakery,Breakfast Spot
19,East Toronto,1,Neighborhood,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
20,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant,Japanese Restaurant
24,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Japanese Restaurant,Burger Joint,Sandwich Place,Thai Restaurant,Gym / Fitness Center,Department Store,Middle Eastern Restaurant,Ice Cream Shop
25,Downtown Toronto,1,Grocery Store,Café,Park,Coffee Shop,Gas Station,Restaurant,Candy Store,Baby Store,Nightclub,Italian Restaurant
30,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Bar,Thai Restaurant,Bakery,Sushi Restaurant,Breakfast Spot,Pizza Place,Salad Place
31,West Toronto,1,Pharmacy,Bakery,Grocery Store,Middle Eastern Restaurant,Music Venue,Park,Coffee Shop,Pool,Café,Brewery


## Summary

A large distinction between cluster label 1 and the other clusters is that cluster 1 generally has coffee shops as either a most common or second most common venue. Meanwhile, the other four clusters shared in having Yoga Studio, Ethiopian, Empanada Restaurant, and Eastern European Restaurant in their top 10.